# Scrapper for guitar selling posts

This script scraps both *The Gear Page* forum and the *Gear4Sale* subreddit for posts selling guitars.
For now the below characteristics are scrapped:
- brand
- price
- vintage
- color
- wood

To do:
- scrap model
- create database
- compute average/median price
- scrap prices from other places ?
- define if a model is "a good buy opportunity"

It then saves the posts in a csv file.

## Persistence

- All the data is persisted to a table in kdb.

- You need to launch localhost with port 41643, username and password is "n1ch3". Then the initialization function will take care of the rest.

- The kdb is assumed to be located at C:/dev/niche/kdb

## Set up

Be sure to have the below path created, or to modify it for the data to be saved where you want

In [445]:
path="C:/dev/niche/guitar/data/"

Be sure to be able to import the below packages

In [446]:
import bs4
import datetime
import pandas as pd
import requests

## Functions

In [491]:
def get_url(section):
    a_s=section.findAll("a")
    for a in a_s:
        if a.has_attr('href'):
            # the first link is the right one
            return a["href"]
        #TODO check link is valid

def validate_string(s):
    """
    Returns True if:
        - s is a string
    &
        - s is not an empty string
    """
    is_s=False
    if type(s)==str:
        if s!="":
            is_s =True
    return is_s

def find_vintage(content):
    result = ""
    content = content.lower()
    # from 1985
    if "from " in content:
        split_on_from =  content.split("from ")
        if len(split_on_from)==2:
            #"guitar from 1985"
            year = split_on_from[1][:4]
            # still check that we are returning a number
            if year.isdigit():
                if check_vintage(year):
                    result = year
        else:
            # "guitar from the best from 1985"
            for i in range(len(split_on_from)):
                year = split_on_from[i][:4]
                # still check that we are returning a number
                if year.isdigit():
                    if check_vintage(year):
                        result = year
    else: 
        potential_years = []
        for i in range(len(content)):
            if len(content[i:])>4:
                four_char = content[i:i+4]
                if four_char.isdigit():
                    if check_vintage(four_char):
                        potential_years.append(four_char)
            if len(potential_years)==1:
                # "guitar blabla 1985"
                result=potential_years[0]
            elif len(potential_years)>1:
                # "guitar made in 1985, bought in 2001"
                result = min(potential_years)
    return result
        # TODO check for "made in", "manufactured in" etc

def find_longest_string_of_digits(s):
    """
    from a string, assuming the first character is a digit, will return the longest number
    eg:
    - "1234 frefr" -> "1234"
    - "j 12345 p" -> ""
    
    :s: string
    """
    i=0
    n = len(s)
    if n>0:
        while (s[i].isdigit() or (s[i]==".") or (s[i]==",")) and (i<n-1):
            i+=1
            if (i==n):
                break
        return s[:i]
    else:
        return ""

def find_price(content):
    result = ""
    content = content.lower()
    if "price is " in content:
        split_on_price_is = content.split("price is ")
        if len(split_on_price_is)==2:
            # price is 300,45
            # find
            p = find_longest_string_of_digits(split_on_price_is[1][find_ind_of_next_digit_in_string(split_on_price_is[1]):])
            if check_price(p):
                result = p
        else:
            ps = []
            # price is 45.00. price is negotiable:
            for i in range(len(split_on_price_is)):
                print(split_on_price_is[i])
                longest_digit = find_longest_string_of_digits(split_on_price_is[i])
                if longest_digit!="":
                    if check_price(longest_digit):
                        ps.append(longest_digit)
            if len(ps)==1:
                result = ps[0]
            elif len(ps)>1:
                result = ps[0]
    for sign in ["usd", "$", "price"]:
        if sign in content:
            split_on_dollar_sign = content.split(sign)
            p = find_longest_string_of_digits(split_on_dollar_sign[1][find_ind_of_next_digit_in_string(split_on_dollar_sign[1]):])
            if p.isdigit():
                if check_price(p):
                    return p
    return result

def check_price(p,lb=1700,ub=50000):
    try:
        if type(p)==str:
            p = float(p.replace(",",""))
        return p>lb  and p <ub
    except Exception as error:
        print(error)
        print(p)

def check_vintage(v):
    """
    vintage shd be between 1900 and this year
    """
    t = datetime.datetime.today()
    if type(v)==str:
        v=int(v)
    return v>1900 and v<t.year+1

def find_ind_of_next_digit_in_string(s):
    """
    Returns the index of the next character which is a digit
    
    eg:
    - "Ah 1" -> 3
    - "2" -> 0
    - "ABC" -> None
    
    """
    for i in range(len(s)):
        if s[i].isdigit():
            return i

def get_brand(content):
    brands = pd.read_csv(path+"guitar_brands.csv")["Brand"].tolist()
    result = ""
    content = content.lower()
    for brand in brands:
        if brand in content:
            return brand
    return result

def get_model(content,brand):
    result = ""
    models = pd.read_csv(path+brand+"_models.csv")["Model"].tolist()
    content = content.lower()
    for model in models:
        if model in content:
            return model
    return result
    
def find_color(s):
    for c in ["blue", "red", "yellow", "pink", "green", "purple", "white", "grey", "black", "beige", "orange"]:
        if c in s:
            return c
        
def get_wood(s):
    """
    Get neckwood
    """
    # TODO: improve this function by getting the wood for the different parts:
    #eg:
    #   Top: Basswood
    #   Body: Basswood
    #   Neck: Mahogany
    #   Fingerboard: Rosewood
    #   Nut: Bone
    
    woods = ["alder","basswod","magahony","swamp ash", "walnut","koa","mapple","rosewood","ebony","wenge"]
    result = ""
    s=s.lower()
    for wood in woods:
        if wood in s:
            return wood
    if "neck wood" in s:
        split_on_neck_wood = s.split("neck wood")
    return result

def find_relevant_information(content,title,url,post_date):
    """
    This is the main function, scrapping the price, vintage etc from title and content and returning a pd.DF row (list)
    """
    # ["brand","model","price","vintage","color","wood","post_date", "post_title","post_content","post_url"]
    new_row = []
    
    # find vintage
    vintage = find_vintage(title)
    if vintage =="":
        vintage = find_vintage(content)
    # find price
    price = find_price(content)

    # If we have price, find brand, model, wood, color
    if price!="":

        # find brand
        brand = get_brand(content)
        if brand=="":
            brand = get_brand(title)
        # find model
        model = get_model(content, brand)
        if model=="":
            model = get_model(content, brand)
        # find color
        color = find_color(content)
        # find wood
        wood = get_wood(content)

        # create a new pd.Df row and insert it if not only there
        new_row=[brand,model,price, vintage,color,wood,post_date,title,content,url,]
        
    return new_row

def scrap_brands_from_wikipedia():
    """
    Scrapp all guitar brands from wikipedia and saves it into a file
    """
    brands = []
    wikipedia_url = "https://en.wikipedia.org/wiki/List_of_guitar_manufacturers"
    wikipedia_rep = requests.get(wikipedia_url)
    wikipedia_soup = bs4.BeautifulSoup(wikipedia_rep.text, 'html.parser')
    lis = wikipedia_soup.findAll("div",{"class": "div-col", "style":"column-width: 22em;"})[0].findAll("li")
    for li in lis:
        a_s = li.findAll("a")
        if len(a_s)>0:
            if a_s[0].has_attr('title'):
                title = str(a_s[0]["title"]).lower()
                # machin (bidule) -> machin
                title = title.split(" (")[0]
                # machin guitars -> machin
                title = title.split(" guitars")[0]
                title = title.split(" guitar")[0]
                # machin music -> machin
                title = title.split(" music")[0]
                brands.append(title)
    pd.DataFrame(brands, columns=["Brand"]).to_csv(path+"guitar_brands.csv", index=False)
    return brands

def kdb_format_year(year):
    year = str(year)
    y = ""
    for char in year:
        if char.isdigit():
            y+=char
    year=y
    if year=="None" or year =="":
        return "0Ni"
    else:
        return year+"i"
    
def insert_scrappedGuitars_into_kdb(df):
    """
    Insert all rows of the data frame into kdb
    Note:The roll to hdb is fully handled in q
    """
    for index, row in df.iterrows():
        post_date = str(row["post_date"])
        brand = str(row["brand"])
        model = str(row["model"])
        price = str(float(row["price"].replace(",","")))
        vintage = kdb_format_year(str(row["vintage"]))
        color = str(row["color"])
        neckWood = str(row["neckWood"])
        pickupType = str(row["pickupType"])
        website = str(row["website"])
        post_title = str(row["post_title"]).replace("\"","").replace(u"\u2019"," ")
        post_title = ''.join([i if ord(i) < 128 else ' ' for i in post_title])
        post_id = "_".join([post_date,website,brand,model,price,vintage]).replace(".","")
        
        kdb_row  = post_date+";"+\
            "`$\""+post_id+"\";"+\
            "`$\""+brand+"\""+";"+\
            "`$\""+model+"\""+";"+\
            price+";"+\
            vintage+";"+\
            "`$\""+color+"\""+";"+\
            "`$\""+neckWood+"\""+";"+\
            "`$\""+pickupType+"\""+";"+\
            "`$\""+website+"\""+";"+\
            "\""+str(row["post_url"])+"\""+";"+\
            "\""+post_title+"\""

        with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
            try:
                X=q('upsert[`scrappedGuitarsToday;('+ kdb_row +')]')
            except Exception as error:
                print(error)
                print('('+ kdb_row +')')
                print("___")
                break
                                
def init_kdb_instance():
    init="""
    scrappedGuitarsToday:([] date:`date$();id:`symbol$();brand:`symbol$();model:`symbol$();price:`float$();vintage:`int$();color:`symbol$();neckWood:`symbol$();pickupType:`symbol$();website:`symbol$();url:();title:());
    recommendedGuitarsToday:([] date:`date$();id:`symbol$());
    
    hdbDirectory : `$"c:/dev/niche/kdb";
    rollToHdb:{[hdbDirectory;data;targetDate]
        show "saving to hdb for date:",string targetDate;
        system "l ",string hdbDirectory;
        $[`scrappedGuitars in tables[];
            [
                show "check there is no id already";
                ids:exec id from select distinct id from scrappedGuitars;
                delete scrappedGuitars from `.;
                dataTargetDate:select from data where date=targetDate, not id in ids;
                ];
            [
                show "Passing here";
                dataTargetDate:select from data where date=targetDate;
                ]
            ];
        show "n:",string count dataTargetDate;
        if[0<count dataTargetDate;
            show "Persisting!";
            `scrappedGuitars set `brand`model xasc delete date from dataTargetDate;
            .Q.dpft[hsym hdbDirectory;targetDate;`brand;`scrappedGuitars]
            ];
        };

    """

    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
            try:
                X=q(init)
            except Exception as error:
                print(error)

def roll_to_HDB_all_dates():
    """
    Rolls all dates of the data in the table to hdb
    """
    roll = "rollToHdb[hdbDirectory;scrappedGuitarsToday;] each exec distinct date from scrappedGuitarsToday;"
    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
        try:
            X=q(roll)
        except Exception as error:
            print(error)
            
def roll_to_HDB_today_only():
    """
    Only saves data for today's date to hdb
    """
    roll = "rollToHdb[hdbDirectory;scrappedGuitarsToday;.z.D]"
    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
        try:
            X=q(roll)
        except Exception as error:
            print(error)
            
def get_average_and_median_price_for_guitar(filters):
    """
    :filters: dict, containing at least keys:"brand" and "model"
    """
    query = "select from scrappedGuitars where date<.z.d"#, brand=`"+filters["brand"]+", model=`$\""+filters["model"]+"\""
    for k in filters.keys():
        query+=","+k+"="+filters[k]
    print(query)
    with qconnection.QConnection(host = "localhost",port=41643, username="n1ch3", password="n1ch3", pandas = True) as q:
        try:
            guitars=q(query)
        except Exception as error:
            print(error)
    mean = guitars["price"].mean()
    median = guitars["price"].median()
    n = len(guitars["price"])
    return [n,mean,median]

## REDDIT

In [456]:
df_reddit=pd.DataFrame(columns=["brand","model","price","vintage","color","wood","post_date","post_title","post_content","post_url"])
k=0
n_pages = 50

print("Starting to scrap The Gear Page, processing the first "+str(n_pages)+" pages.")
after = ""
for page_number in [i+1 for i in range(n_pages)]:
    url = 'https://www.reddit.com/r/Gear4Sale/.json'
    
    if after==None:
        print("No more After pages to scrap, stopping here :)")
        break
    if after!="":
        url += '?&after='+after
    
    r = requests.get(
        url,
        headers={'user-agent': 'Mozilla/5.0'})
    
    data = r.json()['data']
    all_posts =data['children']
    after = data["after"]

    for post in all_posts:
        try:
            title = post["data"]["title"]
            if any(ext in title for ext in ["sell", "wts","Sell","WTS"]):
                title = post["data"]["title"]
                content = post["data"]["selftext"]
                url = post["data"]["url"]
                post_date = datetime.datetime.fromtimestamp(float(post["data"]["created"])).strftime("%Y.%m.%d")
                # create a new pd.Df row and insert it if not only there
                new_row = find_relevant_information(content,title,url,post_date)
                if new_row!=[] and not ((df_reddit['post_title'] == title) & (df_reddit['post_content'] == content)).any():
                    df_reddit.loc[k]=new_row
                    k+=1
        except Exception as error:
            print(error)
    r.close()
    print("... "+str(round(page_number/n_pages*100,2))+"% done")

df_reddit["website"]="Reddit"
df_reddit.to_csv(path+datetime.datetime.now().strftime("%Y_%m_%d")+"_reddit.csv",index=False)

Starting to scrap The Gear Page, processing the first 50 pages.
... 2.0% done
... 4.0% done
... 6.0% done
... 8.0% done
... 10.0% done
... 12.0% done
... 14.0% done
... 16.0% done
... 18.0% done
... 20.0% done
... 22.0% done
... 24.0% done
could not convert string to float: 

... 26.0% done
could not convert string to float: 

... 28.0% done
... 30.0% done
... 32.0% done
... 34.0% done
... 36.0% done
... 38.0% done
... 40.0% done
... 42.0% done
... 44.0% done
... 46.0% done
... 48.0% done
... 50.0% done
could not convert string to float: 

... 52.0% done
... 54.0% done
... 56.0% done
... 58.0% done
... 60.0% done
... 62.0% done
... 64.0% done
... 66.0% done
... 68.0% done
... 70.0% done
... 72.0% done
... 74.0% done
... 76.0% done
... 78.0% done
... 80.0% done
No more After pages to scrap, stopping here :)


## The Gear Page

In [457]:
tgp_base_url = "https://www.thegearpage.net"
tgp_url = "https://www.thegearpage.net/board/index.php?forums/guitar-emporium.22/"
df_tgp=pd.DataFrame(columns=["brand","model","price","vintage","color","wood","post_date","post_title","post_content","post_url"])
k=0
n_pages = 50
print("Starting to scrap The Gear Page, processing the first "+str(n_pages)+" pages.")

for page_number in [i+1 for i in range(n_pages)]:
    url = tgp_url + "page-"+str(page_number)
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    
    for div in soup.findAll("div", {"class":"structItem-cell structItem-cell--main"}):
        try:
            # check that it is a selling post
            if any(tag in div.text.upper() for tag in ["FS" or "FOS" or "WTS" or "SELL" or "SALE"]):
                title = div.findAll("a", {"class":""})[0].text

                # get the post url and from it the content
                post_url = tgp_base_url+div.findAll("a")[1]["href"]
                post_rep = requests.get(post_url)
                post_soup = bs4.BeautifulSoup(post_rep.text, 'html.parser')
                post_date = datetime.datetime.fromtimestamp(float(post_soup.findAll("time",{"class":"u-dt"})[0]["data-time"])).strftime("%Y.%m.%d")
                bbWrappers = post_soup.findAll("div", {"class":"bbWrapper"})
                content = bbWrappers[0].text
                post_rep.close()

                new_row = find_relevant_information(content,title,post_url,post_date)
                if new_row!=[] and not ((df_tgp['post_title'] == title) & (df_tgp['post_content'] == content)).any():
                    df_tgp.loc[k]=new_row
                    k+=1
        except Exception as error:
            print(error)
    r.close()
    print("... "+str(round(page_number/n_pages*100,2))+"% done")

scrapping_results_file_name = path+datetime.datetime.now().strftime("%Y_%m_%d")+"_theGearPage.csv"
print("Succesfully scraped The Gear Page, saving result to file: "+scrapping_results_file_name)

df_tgp["website"]="TheGearPage"
df_tgp.to_csv(scrapping_results_file_name,index=False)

Starting to scrap The Gear Page, processing the first 50 pages.
... 2.0% done
... 4.0% done
... 6.0% done
... 8.0% done
... 10.0% done
... 12.0% done
... 14.0% done
... 16.0% done
... 18.0% done
... 20.0% done
... 22.0% done
... 24.0% done
... 26.0% done
... 28.0% done
... 30.0% done
... 32.0% done
... 34.0% done
... 36.0% done
... 38.0% done
... 40.0% done
could not convert string to float: 

... 42.0% done
... 44.0% done
... 46.0% done
... 48.0% done
... 50.0% done
... 52.0% done
... 54.0% done
... 56.0% done
... 58.0% done
... 60.0% done
... 62.0% done
... 64.0% done
... 66.0% done
... 68.0% done
... 70.0% done
... 72.0% done
... 74.0% done
... 76.0% done
... 78.0% done
... 80.0% done
... 82.0% done
... 84.0% done
... 86.0% done
... 88.0% done
... 90.0% done
... 92.0% done
... 94.0% done
... 96.0% done
... 98.0% done
... 100.0% done
Succesfully scraped The Gear Page, saving result to file: C:/dev/niche/guitar/data/2021_01_19_theGearPage.csv


In [ ]:
init_kdb_instance()

In [461]:
insert_scrappedGuitars_into_kdb(df_tgp)

In [462]:
insert_scrappedGuitars_into_kdb(df_reddit)

In [463]:
roll_to_HDB_all_dates()

In [ ]:
get_average_and_median_price_for_guitar({"brand":"`fender","model":"`$\"\"", "vintage":"2017"})